<a href="https://colab.research.google.com/github/shamilasudalshana/geoai-hydro-minis/blob/main/flood_s1_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# initate and authenticate google Earth Engine and GEE map
! pip -q install earthengine-api geemap xgboost shap scikit-learn streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.4 MB/s eta 0:00:00


In [7]:
# import packages and initiation
import ee, geemap
#geemap.ee_authenticate()
ee.Authenticate()
ee.Initialize(project='flood-s1-gee') # The project name was taken from google cloud earth engine project. it was initiated and needed to activate EE API sepetately, need to be consistant with same google account for this

In [15]:
# importing maps
my_map = geemap.Map(center=[50.55, 7.05], zoom=10) # this is for ahr valley map as an example
my_map

Map(center=[50.55, 7.05], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [12]:
# Polygon RIO creation
roi = my_map.user_roi

if roi is None:
  roi = ee.Geometry.Rectangle([6.85, 50.45, 7.25, 50.75])  # I think this is a fallback bbox

roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              6.85,
              50.75
            ],
            [
              6.85,
              50.45
            ],
            [
              7.25,
              50.45
            ],
            [
              7.25,
              50.75
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [17]:
# Helper to build Sentinel-1 VV composites
def s1_vv_median(start, end, orbit='DESCENDING'):
    col = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(roi)
           .filterDate(start, end)
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.eq('orbitProperties_pass', orbit))
           .select('VV'))
    return col.median().clip(roi)

# --- Set your event dates (example: Germany floods mid-July 2021)
pre    = s1_vv_median('2021-06-15', '2021-07-10')
during = s1_vv_median('2021-07-12', '2021-07-20')

# --- dB difference (during - pre)
dvv = during.subtract(pre).rename('dVV')

# --- Simple flood mask:
# darker during flood (drop in VV) + absolute low VV during event
flood = dvv.lt(-1.5).And(during.lt(-17)).selfMask().rename('flood')

# --- Visualize
my_map.add_layer(pre,    {'min': -25, 'max': 0},    'S1 VV (pre)')
my_map.add_layer(during, {'min': -25, 'max': 0},    'S1 VV (during)')
my_map.add_layer(dvv,    {'min': -4,  'max': 2},    'dVV (during-pre)')
my_map.add_layer(flood,  {'palette': ['#1f78b4']},  'Flood mask')
my_map


Map(bottom=88498.0, center=[50.62006336641878, 6.563301086425782], controls=(WidgetControl(options=['position'…